<a href="https://colab.research.google.com/github/rcdavid1/practice_hospitalwaste/blob/main/hospitalwaste_f25_wide_reduced_cols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

In [ ]:
! pip install gdown

In [ ]:
! wget https://raw.githubusercontent.com/bsheese/225exercises/refs/heads/main/hospital_helper.py

In [ ]:
import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import hospital_helper

file_id = '1BEs2Fa1qJEYWE-HyE3t63PplMi2IbXc9'
output = 'hospital_waste.csv'

gdown.download(id=file_id, output=output, quiet=False)

df = pd.read_csv(output)
df = hospital_helper.clean_variable_names(df)
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df = df.drop(columns = 'unnamed:_0')

focal_variables = ['rmw/apd', 'rmw', 'apd']

# drop all columns that mention specific vendors, such as stryker, medline, cardinal
df = df.loc[:, ~df.columns.str.contains('stryker|medline|cardinal')]

# drop all columns that appear to be constants
df = df.loc[:, ~df.columns.str.contains('rmw/apd_national_median_\\(all\\)|day|available_beds_total|batteries')]

# df.info(show_counts=True, verbose=True)

# address missing values
df = hospital_helper.process_missing(df, verbose=False)


# List of All Columns in Unpivoted Dataset

In [ ]:
for col in list(df.columns):
  print(col)

# Selecting Columns to Keep for Your Analysis
* Any variables you want to use should appear somewhere in the lists below.
* If you do not see what you want, add a list of variables into the list below.

In [ ]:
# this will find variables for you based on part of their name and put them into a list, that you can use in the next step
temp = []

for col in df.columns:
  if "eco2" in col:
    temp.append(col)

temp

In [ ]:
# reorganize columns
cols_listoflists = [
    [# Hospital Identification and Demographics
    'hospital',
    'hospital_abbreviation',
    'city',
    'state',
    'region',
    'eastern_indicator',
    'hospital_size',
    ],

    [# Facility and Operational Metrics
    'square_footage',
    'cleanable_square_footage',
    'payroll_standard_hours_total',
    'purchased_labor_hours_total'
    ],

    [# Regulated Medical Waste (aggregate)
    'rmw',
    'rmw_autoclave',
    'rmw_incineration',
    'rmw/apd',
    'reusable_sharps',
    ],

    ['mt_eco2_(autoclave_-_steam_sterilization)',
    'mt_eco2_(incineration)',
    'mt_eco2_(autoclave_-_etd)',
    'mt_eco2_(rmw_+_haz_pharm)',
    'mt_eco2_(solid_waste)',
    'mt_eco2_(solid_waste_+_rmw_+_haz_pharm)'
    ],

    [# Other
    'hazardous_pharmaceuticals',
    'hazardous:_rcra_pharm',
    'hazardous',
    '5%path/chemo',
    'corrected_path/chemo'
    ],

    [# Recycling
    'rcy',
    'mixed_recycling',
    'recycle_-rd_&_ud',
    'recycle_-_rd_+_ud_+_reprocessing',
    ],
]

# Pivot the data using the columns from above

In [ ]:
# unpack cols into a flat list
flat_list = [item for sublist in cols_listoflists for item in sublist]

# massively reduce columns, add columns back by adding name to lists above
df = df.loc[:, flat_list]

# pivot to shift to wide format using the median function
numeric_cols = df.select_dtypes(include='number').columns
cols_to_pivot = ['hospital', 'hospital_abbreviation',  'city', 'state', 'region', 'hospital_size'] + list(numeric_cols)
df_wide = df[cols_to_pivot].pivot_table(index=['hospital','hospital_abbreviation', 'city', 'state', 'region', 'hospital_size'], aggfunc='median')

# Reindex columns to preserve original order of numeric columns
df_wide = df_wide[numeric_cols].reset_index()

df_wide.head()

# Examples Using `cols_listoflists` to Speed Up Analysis

In [ ]:
for col in cols_listoflists[1:]:
  display(df_wide[col].describe().T)
  print()

In [ ]:
for col in cols_listoflists[1:]:
  display(sns.heatmap(df_wide[col].corr(), cmap="Blues", annot=True))
  plt.show()
  print()